<h3>Build up the network</h3>
Your Genesis cloud API token needs to be set in the environment variable GC_API_TOKEN, the name of your SSH key should be set in the variable GC_SSH_KEY_NAME. If you are sure that you won't accidently push them to github, you can also overwrite them in here.

In [1]:
import os
from create_nodes import *

In [2]:
ssh_key = os.getenv('GC_SSH_KEY_NAME')
api_token = os.getenv('GC_API_TOKEN')

In [3]:
my_client = Client(api_token)
if not my_client.connect():
    print('Connection failed.')
    exit()

In [13]:
gateway = create_gateway(my_client, ssh_key)

Building a image where docker is newly installed.
Instance 9394f598-620f-48e5-8938-5d8cdcf4d96a is starting.
instance is active at 194.61.21.169


In [14]:
workers = create_workers(my_client, ssh_key, gateway.private_ip)

Launching Alice as a worker node on a separate instance.
Worker Alice(68319383-9563-474a-9d8e-71b3e401211a) is starting.


Once the client the client is connected running instances can be recovered based on the id.

In [4]:
gateway = my_client.Instances.get('c32204a9-8b25-4dd1-b9e3-7186ec189988')
workers = [my_client.Instances.get('7c15059b-3717-484b-ad4b-3ac3bf17da11')]

Or based on the name.

In [5]:
gateway = next(my_client.Instances.find({'name':'gateway'}))
instances = list(my_client.Instances.list())
workers = [instance for instance in instances if instance.name != 'gateway']

<h3>Import all the required pysyft</h3>

In [6]:
import torch
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
from syft.workers.node_client import NodeClient
hook = sy.TorchHook(torch)
sy.hook.local_worker.is_client_worker = False

In [9]:
public_grid = PublicGridNetwork(hook, "http://"+gateway.public_ip+":5000")
brutus = NodeClient(hook,"http://"+gateway.public_ip+":3000")
alice = NodeClient(hook,"http://"+nodes[0].public_ip+":3000")

# error messages are caused by bugs in the latest grid-node container.

TypeError: can only concatenate str (not "NoneType") to str

In [10]:
a = torch.tensor([1,2,3])
a_ptr = a.send(alice)
a_ptr = a.send(brutus)
a_ptr.get()

NameError: name 'alice' is not defined

<h3>Destroy the nodes</h3>

In [9]:
for worker in workers:
    my_client.Instances.delete(worker.id)
my_client.Instances.delete(gateway.id)